In [1]:
import datetime
import ifcopenshell
import ifcopenshell.api

In [2]:
model = ifcopenshell.open('test_two_cube_wall.ifc')

In [3]:
walls = model.by_type("IfcWall")

In [4]:
wall1 = walls[0]

In [5]:
wall2 = walls[1]

In [6]:
workplan = ifcopenshell.api.run("sequence.add_work_plan", model, name="Construction Plan")

In [7]:
schedule = ifcopenshell.api.run("sequence.add_work_schedule", model, name="Construction", work_plan=workplan)

In [8]:
def add_task_CONSTRUCTION(model, name, predecessor, work_schedule, object_constructed):
    task = ifcopenshell.api.run("sequence.add_task", model, work_schedule=work_schedule)
    task.Name = name
    task.PredefinedType = "CONSTRUCTION"
    task_time = ifcopenshell.api.run("sequence.add_task_time", model, task=task)
    ifcopenshell.api.run(    
        "sequence.edit_task_time", model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2022, 10, 1), "ScheduleDuration": "P1W"}
    )
    if predecessor:
        ifcopenshell.api.run("sequence.assign_sequence", model, relating_process=predecessor, related_process=task) # defaults to an FS relationship
        
    ifcopenshell.api.run("sequence.assign_product", model, relating_product=object_constructed, related_object=task) #output
    return task

In [9]:
def add_task_DEMOLITION(model, name, predecessor, work_schedule, object_demolated):
    task = ifcopenshell.api.run("sequence.add_task", model, work_schedule=work_schedule)
    task.Name = name
    task.PredefinedType = "DEMOLITION"
    task_time = ifcopenshell.api.run("sequence.add_task_time", model, task=task)
    ifcopenshell.api.run(    
        "sequence.edit_task_time", model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2022, 10, 1), "ScheduleDuration": "P1W"}
    )
    if predecessor:
        ifcopenshell.api.run("sequence.assign_sequence", model, relating_process=predecessor, related_process=task) # defaults to an FS relationship
    
    ifcopenshell.api.run("sequence.assign_process", model, relating_process=task, related_object=object_demolated) #input
    return task

In [10]:
def add_task_MOVE(model, name, predecessor, work_schedule, object_old, object_new):
    task = ifcopenshell.api.run("sequence.add_task", model, work_schedule=work_schedule)
    task.Name = name
    task.PredefinedType = "MOVE"
    task_time = ifcopenshell.api.run("sequence.add_task_time", model, task=task)
    ifcopenshell.api.run(    
        "sequence.edit_task_time", model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2022, 10, 1), "ScheduleDuration": "P1W"}
    )
    if predecessor:
        ifcopenshell.api.run("sequence.assign_sequence", model, relating_process=predecessor, related_process=task) # defaults to an FS relationship
    
    ifcopenshell.api.run("sequence.assign_process", model, relating_process=task, related_object=object_old) #input
    ifcopenshell.api.run("sequence.assign_product", model, relating_product=object_new, related_object=task) #output
    return task

In [11]:
task_con = add_task_CONSTRUCTION(model, "constrution wall", None , schedule, wall1)

In [12]:
task_move = add_task_MOVE(model, "move wall", task_con, schedule, wall1, wall2)

In [13]:
task_demol = add_task_DEMOLITION(model, "demolate wall", task_move, schedule, wall2)

In [14]:
ifcopenshell.api.run("sequence.cascade_schedule", model, task=task_con) # Notice we didn't care much about explicit dates / etc, you can if you want to, but this'll also autocalculate it all for you.
ifcopenshell.api.run("sequence.recalculate_schedule", model, work_schedule=schedule) # Critical path, float, etc

In [15]:
model.write("test_two_cube_wall_4d.ifc")